In [ ]:
!pip install bitsandbytes==0.43.1
!pip install accelerate==0.30.1
!pip install transformers==4.39.3
!pip install gradio==4.29.0

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)

model_id = "google/gemma-1.1-7b-it"
tokenizer = AutoTokenizer.from_pretrained(model_id)

config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="cuda:0",
    trust_remote_code=True,
    quantization_config=config
)

# Zero-Shot Prompt Engineering
- LLM에 예제를 제공하지 않고 응답을 생성하게 하는 기법
- LLM 성능 평가 시, 보통 ㅏㅅ용되는 기법

# Few-Shot Prompt Engineering
- LLM에 몇 가지(Few) 예제를 제공하여 응답을 생성하게 하는 기법
- Zero-Shot 보다 더 나은 성능을 보임
- 예제 개수에 따라 , 1-shot, 3-shot 등으로 불림
- 일정 크기 이상의 LLM에 적용 가능
    ```
    ex)
    Input
    질문 : 너는 리뷰의 긍/부정을 알려주는 로봇이야 "오늘 시켰는데 정말 맛있었어요"
    대답 : "긍정"
    질문 : "별로 였어요"
    대답 : "부정"
    질문 : "한번 더 시켜먹고 싶어요"

    Output
    대답 : "긍정"
    ```

In [ ]:
messages = [
    {"role" : "user", "content": "You are a robot that tells whether a review comment is positive or negative. \nreview comment : 다신 안먹어요"},
    {"role" : "assistant", "content" : "negative"},
    {"role" : "user", "content" : "review comment : 맨날 먹어요 ㅎㅎ"},
    {"role" : "assistant", "content" : "positive"},
    {"role" : "user", "content" : "review comment : 가격이 2배되도 시켜먹겠습니다."}
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

tokenizer.decode(input_ids[0])

In [ ]:
outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    top_p=0.9
)

response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

# Self-Ask Prompt Enginerring
- LLM이 스스로 질문을 던지며 문제 해결을 하면서, 최종적으로 답변을 생성하게 하는 기법

In [ ]:
messages = [
    { "role": "user", "content": """
    You are an English Teacher who teaches Korean Students.
    You always have to explain in the format of a conversation between a student and teacher.
    sententce : 나는 아버지가 방에 들어가는 모습을 보고 많이 후회하고 힘들어했다.
    """ },
    {"role": "assistant", "content": """
    Teacher: What is the verb in the sentence?
    Student: The verb is '후회하고 힘들어했다' which translates to 'regretted and struggled'.
    Teacher: What is the object of the sentence?
    Student: The object is '아버지가 방에 들어가는 모습' which translates to 'the sight of my father entering the room'.
    Teacher: Now, can you try to put it all together in English?
    Student: Yes, the sentence in English would be, "I regretted and struggled a lot after seeing my father entering the room."
    """},
    {"role": "user", "content": "sentence : 어제 밤에 일이 너무 힘들어서 나는 새벽에 깨서 엉엉 울었다."}
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)


In [ ]:
outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    top_p=0.9
)

response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

# Prompt Chaining
- 예시 프롬프트를 제공하고 => 학습
- context를 넣어서 질문, 답변을 생성하도록 함

In [ ]:
import json

# 예시 프롬프트 제공
few_shot_context = "LG전자가 임직원들에게 무료로 사내식당 조식을 제공키로 했다.업계에 따르면 LG전자는 내달 1일부터 3만5000여 명에 달하는 국내 전 사업장 임직원들.."
few_shot_question = "LG전자의 국내 전 사업자 임직원들을 몇 명인가요?"
few_shot_answer = "LG전자의 국내 전 사업자 임직원 수는 약 3만 5000명입니다."

# 질문과 답변을 생성하도록 함
context = "예산군은 2024년도 여름방학 대학생 아르바이트 희망자 40명을 6월 24일부터 26일까지 모집한다고 밝혔다."

messages = [
    {"role" : "user", "content" : f"""You are a robot that generates question and answers using the given context. \n You MUST generate in Korean with JSON. \ncontext : {few_shot_context}"""},
    {"role" : "assistant", "content" : f"{{\"question\" : \"{few_shot_question}\", \"answer\" : \"{few_shot_answer}\"}}"},
    {"role" : "user", "content" : f"context : {context}"}
]

In [ ]:

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

In [ ]:
outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    top_p=0.9
)

response = outputs[0][input_ids.shape[-1]:]
tokenizer.decode(response, skip_special_tokens=True)

# '{"question": "예산군이 2024년도 여름방학 대학생 아르바이트 희망자를 언제 모집할까요?", "answer": "예산군은 2024년도 여름방학 대학생 아르바이트 희망자를 6월 24일부터 26일까지 모집합니다."}'


In [ ]:
question = json.loads(tokenizer.decode(response, skip_special_tokens=True))['question']
answer= json.loads(tokenizer.decode(response, skip_special_tokens=True))['answer']

In [ ]:
question
# '예산군이 2024년도 여름방학 대학생 아르바이트 희망자를 언제 모집할까요?'

In [ ]:
answer = "예산군은 2024년도 여름방학 대학생 아르바이트 희망자를 6월 24일부터 25일까지 모집합니다." 

In [ ]:
context
# '예산군은 2024년도 여름방학 대학생 아르바이트 희망자 40명을 6월 24일부터 26일까지 모집한다고 밝혔다.'

In [ ]:
messages = [
    {"role": "user", "content" :f"""You are a robot that judges whether the answer is correct by looking at the content, question, and answer. Judge and explain why you think so in Korean.
    \ncontext : {few_shot_context}\nquestion : {few_shot_question}\nanswer : {few_shot_answer}"""},
    {"role": "assistant", "content" : """{{"result" : "True", "reason" : "context에 35000명이라고 명시가 되어 있기 때문입니다"}}"""},
    {"role": "user", "content" : f"context : {context}\nquestion : {question}\nanswer : {answer}"}
]

In [ ]:

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

In [ ]:
outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    top_p=0.9
)

response = outputs[0][input_ids.shape[-1]:]
tokenizer.decode(response, skip_special_tokens=True)

# '{{"result" : "False", "reason" : "문제에서 제공된 답은 \'6월 24일부터 26일까지\'입니다. 하지만 질문은 \' 언제\'라고 물고 있습니다."}}'